In [1]:
#!/usr/bin/python
# -*- coding: UTF-8 -*-
from __future__ import division
import csv
import random
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import *
import time

In [2]:


def genTransactions(inpath):
    """ Generates a list of transactions, where each transaction is a set of items. 
    """
    ids, ts = list(), list()
    with open(inpath, 'r') as f:
        for line in f:
            i, t = line.strip().split('\t')
            ids.append(i)
            ts.append(set(t.split(',')))
    return ids, ts


def genCand(F):
    C = list()
    def check(curr):
        for i in curr:
            tmp = curr - {i}
            if not tmp in F: return False
        return True
    for i in xrange(len(F)-1): 
        for j in xrange(i+1, len(F)):
            print F[i], F[j], len(F), '============'
            curr = F[i].union(F[j])
            if (not curr in C) and len(curr)==len(F[i])+1 and check(curr):
                C.append(frozenset(curr))
    return C



minsup = 0.001 #minimum support ratio
niters = 2 #number of iterations
results = dict() #results[k] saves k-item frequent set

inpath = 'data1/transactions.txt'
ids, T = genTransactions(inpath)
del ids

sc = SparkContext()

# def apriori(T, minsup):
n = len(T)
print n 
trans = sc.parallelize(T)
trans_set = trans.map(lambda x: set(x)).cache()

# Gets one-item candidate set
trans_set_one = trans.flatMap(lambda x: frozenset(x)).cache()
trans_set_one = trans_set_one.map(lambda x: set(x)).cache()



# Calculates support ratios
trans_set_values = trans_set_one.map(lambda x: (set(x), 1/float(n))).reduceByKey(lambda x,y: x+y)
# Gets one-item frequent set
freq_set_values = trans_set_values.filter(lambda x: x[1]>=minsup)
freq_set_one = freq_set_values.map(lambda x:x[0]).collect()



trans_set_bc = sc.broadcast(trans_set.collect())


c = 1
results[c] = freq_set_one
while c < niters+1:
    
    print ('== %s th iteration ==' % str(c))
    
    cand_set = genCand(results[c])
    freq_set_values = sc.parallelize(cand_set).map(lambda x: (set(x), len([t for t in trans_set_bc.value if x.issubset(t)])/n)).filter(lambda x:x[1]>=minsup).cache()
    freq_set = freq_set_values.map(lambda x: x[0]).collect()
    c += 1
    results[c] = freq_set 
print results 
# return results








IOError: [Errno 2] No such file or directory: 'data1/transactions.txt'